In [1]:
#Access your secret keys in Python via:
from google.colab import userdata

BUCKET_NAME =userdata.get('cvBucket')
ACCESS_KEY= userdata.get('accessKey')
SECRET_KEY= userdata.get('secretKey')

In [2]:
!pip install boto3 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 896.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 8.1 MB/s eta 0:00:00


In [3]:
import boto3

# Object to access S3 resources.
s3 = boto3.resource('s3', aws_access_key_id = ACCESS_KEY, aws_secret_access_key= SECRET_KEY)

## Looking out for all avialable dataset within that bucket

In [4]:
# List the files in the specified S3 bucket
def list_files_in_bucket(bucket_name):
    bucket = s3.Bucket(bucket_name)

    for obj in bucket.objects.all():
        print(obj.key)

# Call the function to list files
list_files_in_bucket(BUCKET_NAME)

Test_Raw_images.zip
dataset.pkl
face_detector.h5
raw_image_files.zip
test_dataset.pkl


In [5]:
#test_dataset.pkl is the file we should download

import botocore
# save the filename in KEY variable
KEY = 'test_dataset.pkl' # replace with your object key

try:
  # we are trying to download file from s3 with name `raw_image_files.zip` stored in varaible KEY
  # to colab dir with name `raw_image_files.zip`
  s3.Bucket(BUCKET_NAME).download_file(KEY, 'test_dataset.pkl')

except botocore.exceptions.ClientError as e:
  if e.response['Error']['Code'] == "404":
    print("The object does not exist.")
  else:
    raise

In [6]:
## Un-pickle
import pickle
with open('/content/test_dataset.pkl', 'rb') as f:
    test_df = pickle.load(f)

In [ ]:
test_df.head()

,file_path,normalized_image,label
0,/content/Test_Raw_images/acceptable/images (9)...,"[[[0.6901960784313725, 0.7333333333333333, 0.7...",acceptable
1,/content/Test_Raw_images/acceptable/download (...,"[[[0.6274509803921569, 0.7372549019607844, 0.8...",acceptable
2,/content/Test_Raw_images/acceptable/images (6)...,"[[[0.08627450980392157, 0.06666666666666667, 0...",acceptable
3,/content/Test_Raw_images/acceptable/download (...,"[[[0.47843137254901963, 0.49019607843137253, 0...",acceptable
4,/content/Test_Raw_images/acceptable/images (8)...,"[[[0.1607843137254902, 0.3803921568627451, 0.5...",acceptable


In [7]:
test_df['label'].value_counts()

label
alarming      28
acceptable    27
Name: count, dtype: int64

In [8]:
type(test_df['normalized_image'][0]),test_df['normalized_image'][0].shape

(numpy.ndarray, (224, 224, 3))

#Download Model

In [9]:
#face_detector.h5 is the file we should download

import botocore
# save the filename in KEY variable
KEY = 'face_detector.h5' # replace with your object key

try:
  # we are trying to download file from s3 with name `raw_image_files.zip` stored in varaible KEY
  # to colab dir with name `raw_image_files.zip`
  s3.Bucket(BUCKET_NAME).download_file(KEY, 'face_detector.h5')

except botocore.exceptions.ClientError as e:
  if e.response['Error']['Code'] == "404":
    print("The object does not exist.")
  else:
    raise

#Load Model

In [10]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('face_detector.h5')

In [11]:
import numpy as np

test_images=np.stack(test_df['normalized_image'])

In [12]:
# Make predictions
predictions = model.predict(test_images)

# If it's a binary classification model, predictions will be probabilities
# Convert probabilities to class labels (0 or 1)
predicted_labels = (predictions > 0.5).astype(int)


2/2 [==============================] - 30s 13s/step


In [16]:
predicted_labels

#predictions


array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]])

In [17]:
test_df['predicted_labels']=predicted_labels

In [18]:
# Define the mapping of folder names to labels
label_mapping = {0:'acceptable',1:'alarming'}
test_df['predicted_labels'] =test_df['predicted_labels'].map(label_mapping)

test_df

,file_path,normalized_image,label,predicted_labels
0,/content/Test_Raw_images/acceptable/images (9)...,"[[[0.6901960784313725, 0.7333333333333333, 0.7...",acceptable,alarming
1,/content/Test_Raw_images/acceptable/download (...,"[[[0.6274509803921569, 0.7372549019607844, 0.8...",acceptable,alarming
2,/content/Test_Raw_images/acceptable/images (6)...,"[[[0.08627450980392157, 0.06666666666666667, 0...",acceptable,alarming
3,/content/Test_Raw_images/acceptable/download (...,"[[[0.47843137254901963, 0.49019607843137253, 0...",acceptable,alarming
4,/content/Test_Raw_images/acceptable/images (8)...,"[[[0.1607843137254902, 0.3803921568627451, 0.5...",acceptable,alarming
5,/content/Test_Raw_images/acceptable/images (12...,"[[[0.39215686274509803, 0.41568627450980394, 0...",acceptable,alarming
6,/content/Test_Raw_images/acceptable/download (...,"[[[0.611764705882353, 0.5843137254901961, 0.54...",acceptable,acceptable
7,/content/Test_Raw_images/acceptable/1.jpg,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",acceptable,acceptable
8,/content/Test_Raw_images/acceptable/images (7)...,"[[[0.14901960784313725, 0.15294117647058825, 0...",acceptable,alarming
9,/content/Test_Raw_images/acceptable/images (13...,"[[[0.8431372549019608, 0.7294117647058823, 0.4...",acceptable,acceptable


In [19]:
# Assuming you have true labels for the test images
#true_labels = [...]  # Replace with your actual test labels

# Calculate accuracy
accuracy = np.mean(test_df['predicted_labels'] == test_df['label'])
print(f'Accuracy: {accuracy * 100:.2f}%')

# Optionally, print the predictions
#for img_path, pred, true_label in zip(test_image_paths, predicted_labels, true_labels):
 #   print(f'Image: {img_path}, Predicted: {pred}, True: {true_label}')


Accuracy: 69.09%
